In [35]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn import svm
from pandas.tseries.offsets import MonthEnd
%matplotlib inline

In [3]:
%run -i financial_sector_functions.py

/home/luke/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
def return_function(df):
    if type(df)!=pd.DataFrame:
        df = pd.DataFrame(df)
    out_df = (df.shift(-1)-df)/df
    return out_df

In [8]:
data = pd.read_csv("financial_sector_clean_return_space_Dec_13.csv")
data.index = pd.DatetimeIndex(data.iloc[:,[0]].values.flatten())
data = data.iloc[:,1:]

sp500 = pd.read_csv('/home/luke/workspace/Sector-Analysis/data/CFRM521_final_project/SPX500.csv')
sp500.index = pd.DatetimeIndex(sp500.iloc[:,[0]].values.flatten())
sp500 = sp500.iloc[:,[1]]
sp500 = return_function(sp500)
sector_return = data.iloc[:,[0]]
x_data = data.iloc[:,1:]
sector_return,x_data = model_data(sector_return,x_data)
sector_return,sp500 = model_data(sector_return,sp500)

sector_beat_benchmark = pd.DataFrame((sector_return.values > sp500.values)*1,index = sector_return.index,columns = ['IFY beat benchmark'])

In [28]:
result_df = pd.DataFrame()
for gamma_value in map(lambda x: 10**(float(x)),np.arange(-3,4)):
    for c in np.linspace(0.1,10,20):
        svc_model = svm.SVC(C=c,gamma=gamma_value)
        svc_model.fit(x_data, sector_beat_benchmark)
        result_df.loc[gamma_value,c]=(svc_model.score(X=x_data,y=sector_beat_benchmark))

In [36]:
def svc_fit_function(x,y,nunmber_of_years_lookback,month_forecast,c,gamma_value):
#     y,benchmark = model_data(y,benchmark)
    logistic_result_dict = {}
    coef_all_df = pd.DataFrame()
    accuracy_dict = {}
    y_prediction_dict ={}
    number_of_runs = 0.0
    
    coef_pval_all = pd.DataFrame()
    coef_all = pd.DataFrame()
    for dt_index in x.index[x.index>=x.index.min()+pd.DateOffset(years = nunmber_of_years_lookback)][:-(month_forecast-1)]:
        begin_index,end_index = dt_index+pd.DateOffset(years = -nunmber_of_years_lookback)+MonthEnd(0),dt_index+MonthEnd(-1)
        selection = (x.index>=begin_index) & (x.index <= end_index)
        spliced_x = x[selection]
        spliced_y = y[selection]

        assert(len(spliced_x.index)==nunmber_of_years_lookback*12)
        model = sm.OLS(spliced_y,spliced_x)
        result = model.fit()
        svc_model = svm.SVC(C=c,gamma=gamma_value)
        svc_model.fit(spliced_x, spliced_y)

        test_begin_index,test_end_index = dt_index+pd.DateOffset(months=+1)+MonthEnd(-1),dt_index+pd.DateOffset(months=+month_forecast)+MonthEnd(-1)
        test_selection = (x.index>=test_begin_index) & (x.index<=test_end_index)
        test_x = x[test_selection]
        test_y = y[test_selection]
        
        assert(len(test_y)==month_forecast)
        y_prediction = pd.DataFrame(svc_model.predict(test_x),index = test_x.index,columns = ['predict'])
        y_prediction_dict[dt_index] = y_prediction
        
        accuracy_df = pd.DataFrame((y_prediction.values==test_y.values)*1,index=test_y.index)
        accuracy_dict[dt_index] = accuracy_df
        number_of_runs +=1

    mean_accuracy_list = []
    for key,value in accuracy_dict.iteritems():
        mean_accuracy_list.append((value.values.flatten()))
    average_accuracy = np.mean(mean_accuracy_list)
    return average_accuracy

In [43]:
month = 3
year = 10

def svc_choose_year_month(month,year):
    result_df = pd.DataFrame()
    for gamma_value in map(lambda x: 10**(float(x)),np.arange(-3,4)):
        for c in np.linspace(0.1,10,20):
            accuracy = svc_fit_function(x=x_data,y=sector_beat_benchmark,
                             nunmber_of_years_lookback=year,month_forecast=month,c=c,gamma_value=gamma_value)
            result_df.loc[gamma_value,c] = accuracy
    selected_C, selected_lambda = result_df.max(axis=0).idxmax(),result_df.max(axis=1).idxmax()
    max_acf = np.max(result_df.values)
    return max_acf,selected_C,selected_lambda

In [46]:
accuracy_df = pd.DataFrame()
C_df = pd.DataFrame()
lambda_df = pd.DataFrame()

for month in np.arange(3,12+1):
    for year in np.arange(3,11):
        max_acf,selected_C,selected_lambda = svc_choose_year_month(month,year)
        accuracy_df.loc[str(year)+" year",str(month)+" month"]=max_acf
        C_df.loc[str(year)+" year",str(month)+" month"]=selected_C
        lambda_df.loc[str(year)+" year",str(month)+" month"]=selected_lambda

In [47]:
np.max(accuracy_df.values),accuracy_df

(0.6216216216216216,
           3 month   4 month   5 month   6 month   7 month   8 month   9 month  \
 3 year   0.584906  0.580696  0.577070  0.577991  0.575115  0.573052  0.574437   
 4 year   0.585034  0.587329  0.584828  0.581019  0.577423  0.579225  0.576832   
 5 year   0.590123  0.587687  0.581955  0.582071  0.582334  0.582692  0.579673   
 6 year   0.612466  0.608607  0.606612  0.600000  0.593037  0.591102  0.586895   
 7 year   0.609610  0.611364  0.603670  0.591049  0.580774  0.573113  0.569312   
 8 year   0.599327  0.599490  0.595876  0.592014  0.584962  0.583777  0.583035   
 9 year   0.601533  0.604651  0.607059  0.613095  0.614458  0.614329  0.611797   
 10 year  0.617778  0.621622  0.619178  0.618056  0.615694  0.617857  0.615137   
 
          10 month  11 month  12 month  
 3 year   0.573026  0.570138  0.570556  
 4 year   0.574286  0.571615  0.570652  
 5 year   0.577344  0.574087  0.570767  
 6 year   0.583621  0.581818  0.578947  
 7 year   0.565385  0.562224  0.55

In [48]:
1

1